In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [41]:
import os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import random

random_dir = random.choice(os.listdir('/kaggle/input/sports-classification'))
targets = ['test','train','valid']
random_dir2 = '/kaggle/input/sports-classification'+'/'+random_dir
while random_dir not in targets:
    random_dir = random.choice(os.listdir('/kaggle/input/sports-classification'))
    random_dir2 = '/kaggle/input/sports-classification'+'/'+random_dir
random_class = random.choice(os.listdir(random_dir2))
random_class = random_dir2 + '/'+random_class
random_image = random.choice(os.listdir(random_class))
random_image = random_class + '/' + random_image

img = mpimg.imread(random_image)
plt.imshow(img)

In [42]:
import tensorflow as tf

train_dir = '/kaggle/input/sports-classification/train/'
valid_dir = '/kaggle/input/sports-classification/valid/'
test_dir = '/kaggle/input/sports-classification/test/'
test2_dir = '/kaggle/input/sports-classification/images to predict/'

In [43]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                image_size=(224,224),
                                                                batch_size=32,
                                                                label_mode='categorical')
valid_data = tf.keras.preprocessing.image_dataset_from_directory(valid_dir,
                                                                image_size=(224,224),
                                                                batch_size=32,
                                                                label_mode='categorical')
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                image_size=(224,224),
                                                                batch_size=32,
                                                                label_mode='categorical')

In [44]:
data_augmentation = tf.keras.models.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomWidth(0.2),
    tf.keras.layers.RandomHeight(0.2),
    tf.keras.layers.RandomZoom(0.2)
])

In [56]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False
i = tf.keras.layers.Input(shape=(224,224,3))
x = data_augmentation(i)
x = base_model(x,training=False)
x = tf.keras.layers.GlobalMaxPool2D()(x)
x = tf.keras.layers.Dense(100,activation='softmax')(x)
model = tf.keras.Model(inputs=(i),outputs=(x))


In [46]:
model.compile(optimizer=tf.keras.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [49]:
history = model.fit(train_data, validation_data= val_generator,validation_steps=int(0.15*len(val_generator)),
                    epochs = 5)

In [50]:
model.summary()

In [57]:
base_model.trainable = True
for layer in base_model.layers[:-10]:
    layer.trainable = False


In [58]:
model.summary()

In [60]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [61]:
history_fine_tuned = model.fit(train_data,validation_data=(valid_data),epochs=5,
                              validation_steps=(0.15*len(valid_data)))

In [62]:
preds = model.evaluate(test_data)

In [99]:
import matplotlib.image as mpimg
images = []
for i in os.listdir(test2_dir):
    img = mpimg.imread(os.path.join(test2_dir,i))
    images.append(img)
images[0].shape

In [100]:

images = (y for y in images)
next(images)

In [101]:
pred = model.predict(images)

In [103]:
preds = []
for i in pred:
    p = np.max(np.round(i))
    p2 = np.round(i)
    preds.append(np.where(p2==p))

In [104]:
len(pred)

In [105]:
preds